# Q-Learning on FrozenLake (Gymnasium)
We will use the FrozenLake-v1 environment from Gymnasium and solve it with Q-learning.


## Imports and Helpers
We import the libraries, set a random seed, and make a helper for moving averages.


In [2]:
import numpy as np
import gymnasium as gym
from collections import deque
import matplotlib.pyplot as plt

# For reproducibility
SEED = 0
np.random.seed(0)

def moving_average(x, window=100):
    if len(x) < window:
        return np.array(x, dtype=float)
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[window:] - cumsum[:-window]) / float(window)

ARROWS = {0: "←", 1: "↓", 2: "→", 3: "↑"}

## Environment
We use the classic **4x4 FrozenLake** environment.  
- Start = top left  
- Goal = bottom right  
- Holes = fall in, episode ends  
- Slippery ice makes actions stochastic


In [ ]:
# Classic 4x4 FrozenLake
# +1 for reaching the goal, 0 otherwise
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True)
n_states = env.observation_space.n
n_actions = env.action_space.n

print("Observation space:", n_states)
print("Action space:", n_actions)

Observation space: 16
Action space: 4


## Q-table and Hyperparameters
- **Q-table**: stores values for each state-action pair  
- **α (alpha)**: learning rate  
- **γ (gamma)**: discount factor  
- **ε (epsilon)**: exploration rate


In [ ]:
Q = np.zeroes((n_states, n_actions))

# Hyperparameters
alpha = 0.2
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.05
target_fraction = 0.95

episodes = 60000
max_steps_per_episode = 100

epsilon_decay = (epsilon_min/epsilon) ** (1/(episodes*target_fraction))

# Tracking
episode_rewards = []
episode_success = []
smoothed_success = []

## Epsilon-Greedy Strategy
At each step:
- With probability ε: pick a random action (explore)  
- With probability 1-ε: pick the best-known action (exploit)


In [ ]:
def epsilon_greedy_action(state, epsilon):
    if np.random.rand() < epsilon:
        return env.action_space.sample() # explore
    else:
        return int(np.argmax(Q[state]))

## Training with Q-learning
For each episode:
1. Start at the beginning  
2. Choose actions with ε-greedy  
3. Update the Q-table using the Q-learning rule  
4. Decay ε over time  

We repeat for many episodes so the agent learns a good policy.


In [ ]:
for ep in range(episodes):
    state, info = env.reset()
    total_reward = 0

    for t in range(max_steps_per_episode):
        action = epsilon_greedy_action(state, epsilon)

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Q-learning update
        best_next = np.max(Q[next_state])
        td_target = reward + gamma * best_next
        td_error = td_target - Q[state, action]
        Q[state, action] += alpha * td_error 

        state = next_state
        total_reward += reward
        if done:
            break

    # Bookkeeping
    episode_rewards.append(total_reward)
    episode_success.append(1 if total_reward > 0 else 0)

    # Decay
    epsilon = max(epsilon_min, epsilon * epsilon_decay)


## Learning Curve
We plot the moving average of success rate across episodes.  
This shows if the agent is improving at reaching the goal.


In [ ]:
# Compute moving average success (window=100 and 1000)
smoothed_success_100 = moving_average(episode_success, window=100)
smoothed_success_1000 = moving_average(episode_success, window=1000)

plt.figure(figsize=(6, 4))
plt.plot(smoothed_success_100, label="100", alpha=0.6)
plt.plot(smoothed_success_1000, label="1000", color="orange")
plt.title("FrozenLake: Moving Average Success Rate")
plt.xlabel("Episode")
plt.ylabel("Success Rate")
plt.grid(True)
plt.legend()
plt.show()